# Interaktivní vizualizace spektra rentgenového záření

Tento notebook interaktivně demonstruje vznik **nevyfiltrovaného spektra** rentgenového záření. Umožňuje měnit **materiál anody** a **napětí na rentgence (kVp)** a sledovat jejich vliv na výsledné spektrum, jeho **kvalitu** a **kvantitu**.

## Fyzikální pozadí ⚛️

Rentgenové spektrum se skládá ze dvou hlavních složek:

1.  **Brzdné záření (Bremsstrahlung):** Vytváří **spojité spektrum**. Vzniká zpomalením elektronů v elektrickém poli jader anody. Jeho maximální energie ($E_{max}$) je přímo dána napětím na rentgence (kVp). Jeho intenzita je úměrná protonovému číslu anody (Z).
2.  **Charakteristické záření:** Vytváří **diskrétní (čárové) spektrum**. Vzniká, když elektron z katody vyrazí elektron z vnitřní slupky atomu anody a na jeho místo přeskočí elektron z vyšší slupky. Energie emitovaného fotonu je pak rovna rozdílu vazebných energií a je specifická pro daný materiál.

* **Kvantita záření:** Celkový počet fotonů (plocha pod křivkou spektra). Závisí na Z, kVp a mAs.
* **Kvalita záření:** Průměrná energie fotonů, která určuje pronikavost svazku. Závisí primárně na kVp a filtraci.

## Jak používat notebook?

1.  **Spusťte buňku s kódem** (klávesová zkratka `Shift + Enter`).
2.  **Použijte posuvník `kVp`** pro změnu napětí na rentgence.
3.  **Vyberte materiál anody** z rozbalovacího menu pro porovnání různých spekter.

In [ ]:
# Import potřebných knihoven
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, Dropdown, IntSlider
from IPython.display import display
from scipy.stats import norm

# Data pro různé materiály anody: Z, K-hrana, Kα a Kβ energie [keV]
anode_materials = {
    'Molybden (Mo)': {
        'Z': 42,
        'k_edge': 20.0,
        'k_alpha': 17.5,
        'k_beta': 19.6
    },
    'Rhodium (Rh)': {
        'Z': 45,
        'k_edge': 23.2,
        'k_alpha': 20.2,
        'k_beta': 22.7
    },
    'Volfram (W)': {
        'Z': 74,
        'k_edge': 69.5,
        'k_alpha': 59.3,
        'k_beta': 67.2
    },
    'Měď (Cu)': {
        'Z': 29,
        'k_edge': 9.0,
        'k_alpha': 8.0,
        'k_beta': 8.9
    }
}

# Hlavní funkce pro vykreslení grafu
def plot_xray_spectrum(anode_name, kvp):
    material = anode_materials[anode_name]
    Z = material['Z']
    
    # Energetická osa až do 150 keV
    E = np.linspace(1, 150, 1000)
    
    # 1. Výpočet brzdného záření (Kramerova aproximace)
    brems = Z * (kvp - E)
    brems[E > kvp] = 0 # Intenzita je nulová pro energie vyšší než kVp
    brems[E < 0] = 0
    
    total_spectrum = brems
    
    # 2. Přidání charakteristického záření (pokud je kVp dostatečně vysoké)
    if kvp > material['k_edge']:
        # Intenzita a šířka píků
        peak_intensity_alpha = np.max(brems) * 1.5
        peak_intensity_beta = np.max(brems) * 0.5
        peak_width = 0.5 # Šířka v keV
        
        char_alpha = norm.pdf(E, material['k_alpha'], peak_width) * peak_intensity_alpha
        char_beta = norm.pdf(E, material['k_beta'], peak_width) * peak_intensity_beta
        total_spectrum = brems + char_alpha + char_beta

    # 3. Výpočet kvality a kvantity
    # Kvantita = plocha pod křivkou (suma intenzit)
    quantity = np.sum(total_spectrum)
    
    # Kvalita = průměrná energie spektra
    if quantity > 0:
        quality = np.average(E, weights=total_spectrum)
    else:
        quality = 0
    
    # Vykreslení grafu
    fig, ax = plt.subplots(figsize=(12, 7))
    
    ax.plot(E, total_spectrum, color='royalblue', linewidth=2.5, label='Celkové spektrum')
    ax.fill_between(E, total_spectrum, color='skyblue', alpha=0.3)
    
    # Vyznačení kvality (průměrné energie)
    ax.axvline(quality, color='crimson', linestyle='--', linewidth=2, label=f'Kvalita (průměrná E): {quality:.1f} keV')

    # Popisky a titulek
    ax.set_title(f'Nevyfiltrované spektrum pro anodu: {anode_name} (Z={Z}) při {kvp} kVp', fontsize=16)
    ax.set_xlabel('Energie fotonu [keV]', fontsize=12)
    ax.set_ylabel('Relativní intenzita', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.set_xlim(0, 150)
    ax.set_ylim(bottom=0)
    
    # Textové pole s hodnotou kvantity
    quantity_text = f'Kvantita (plocha pod křivkou): {quantity/1e5:.2f} [rel. jedn.]'
    ax.text(0.55, 0.9, quantity_text, transform=ax.transAxes, fontsize=12, 
            verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))
    
    ax.legend(fontsize=11)
    plt.show()

In [2]:
# Vytvoření a zobrazení interaktivních prvků
kvp_slider = IntSlider(
    value=80, min=20, max=150, step=1, 
    description='Napětí (kVp):', 
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

anode_dropdown = Dropdown(
    options=list(anode_materials.keys()),
    value='Volfram (W)',
    description='Materiál anody:',
    style={'description_width': 'initial'},
    layout={'width': '50%'}
)

ui = widgets.VBox([kvp_slider, anode_dropdown])
out = widgets.interactive_output(plot_xray_spectrum, {'anode_name': anode_dropdown, 'kvp': kvp_slider})

display(ui, out)

Output()

## Klíčová zjištění a experimenty 📈

* **Vliv napětí (kVp):**
    * Zkuste plynule zvyšovat `kVp`. Všimněte si, že se **zvyšuje maximální energie** (křivka se posouvá doprava).
    * Současně se dramaticky **zvyšuje kvantita** (celková plocha pod křivkou) a také **kvalita** (červená čára se posouvá doprava).

* **Vliv materiálu anody (Z):**
    * Porovnejte `Měď (Cu, Z=29)` a `Volfram (W, Z=74)` při stejném kVp (např. 100 kVp).
    * U volframu je **kvantita** (intenzita brzdného záření) výrazně vyšší.
    * **Charakteristické píky** se nacházejí na úplně jiných, pro daný prvek specifických, energiích.

* **Podmínka pro charakteristické záření:**
    * Vyberte `Volfram (W)`, jehož K-hrana je ~70 keV.
    * Nastavte `kVp` na hodnotu **pod 70** (např. 65 kVp). Charakteristické píky zmizí! Elektrony z katody nemají dostatek energie, aby vyrazily elektrony z K-slupky.
    * Jakmile `kVp` zvýšíte **nad 70**, píky se skokově objeví.